In [ ]:
import mysql.connector
from tkinter import *
from tkinter import messagebox, ttk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress

# ------------------- Database Connection -------------------
conn = mysql.connector.connect(
    host="localhost",
    user="root",  # change if needed
    password="",  # your MySQL password
    database="health_dashboard"
)
cursor = conn.cursor()

# ------------------- Root Window -------------------
root = Tk()
root.title("AI-Powered Personal Health Dashboard")
root.geometry("1400x800")
root.configure(bg="#f0f4f7")  # modern light background

# ------------------- Header -------------------
header = Frame(root, bg="#1a75ff", height=80)
header.pack(fill=X)

# Logo
try:
    logo_img = PhotoImage(file="logo.png")  # replace with your logo file
    logo_label = Label(header, image=logo_img, bg="#1a75ff")
    logo_label.pack(side=LEFT, padx=20, pady=10)
except:
    pass

Label(header, text="AI-Powered Personal Health Dashboard", font=("Helvetica", 26, "bold"),
      bg="#1a75ff", fg="white").pack(side=LEFT, padx=20)

# ------------------- Variables -------------------
id_var = StringVar()
name_var = StringVar()
age_var = StringVar()
gender_var = StringVar()
weight_var = StringVar()
height_var = StringVar()
steps_var = StringVar()
calories_var = StringVar()
heart_rate_var = StringVar()
sleep_var = StringVar()

total_var = StringVar()
avg_steps_var = StringVar()
avg_cal_var = StringVar()
avg_hr_var = StringVar()
avg_sleep_var = StringVar()

# ------------------- Functions -------------------
def add_patient():
    try:
        sql = "INSERT INTO patient_health_data (name, age, gender, weight, height, steps, calories, heart_rate, sleep_hours) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        values = (name_var.get(), age_var.get(), gender_var.get(), weight_var.get(),
                  height_var.get(), steps_var.get(), calories_var.get(), heart_rate_var.get(), sleep_var.get())
        cursor.execute(sql, values)
        conn.commit()
        messagebox.showinfo("Success", "Patient added successfully!")
        fetch_patients()
        update_summary()
    except Exception as e:
        messagebox.showerror("Error", str(e))

def update_patient():
    try:
        sql = "UPDATE patient_health_data SET name=%s, age=%s, gender=%s, weight=%s, height=%s, steps=%s, calories=%s, heart_rate=%s, sleep_hours=%s WHERE id=%s"
        values = (name_var.get(), age_var.get(), gender_var.get(), weight_var.get(),
                  height_var.get(), steps_var.get(), calories_var.get(), heart_rate_var.get(), sleep_var.get(), id_var.get())
        cursor.execute(sql, values)
        conn.commit()
        messagebox.showinfo("Success", "Patient updated successfully!")
        fetch_patients()
        update_summary()
    except Exception as e:
        messagebox.showerror("Error", str(e))

def delete_patient():
    try:
        sql = "DELETE FROM patient_health_data WHERE id=%s"
        cursor.execute(sql, (id_var.get(),))
        conn.commit()
        messagebox.showinfo("Success", "Patient deleted successfully!")
        fetch_patients()
        update_summary()
    except Exception as e:
        messagebox.showerror("Error", str(e))

def fetch_patients():
    for row in patient_table.get_children():
        patient_table.delete(row)
    cursor.execute("SELECT * FROM patient_health_data")
    rows = cursor.fetchall()
    for row in rows:
        patient_table.insert('', END, values=row)

def select_patient(event):
    selected = patient_table.focus()
    values = patient_table.item(selected, 'values')
    if values:
        id_var.set(values[0])
        name_var.set(values[1])
        age_var.set(values[2])
        gender_var.set(values[3])
        weight_var.set(values[4])
        height_var.set(values[5])
        steps_var.set(values[6])
        calories_var.set(values[7])
        heart_rate_var.set(values[8])
        sleep_var.set(values[9])

def show_charts():
    for widget in chart_frame.winfo_children():
        widget.destroy()
        
    cursor.execute("SELECT steps, calories, heart_rate, sleep_hours FROM patient_health_data")
    data = cursor.fetchall()
    if not data:
        messagebox.showwarning("No Data", "No data to show charts!")
        return
    
    steps, calories, heart_rate, sleep_hours = zip(*data)
    sns.set(style="whitegrid", palette="muted")
    
    fig, axs = plt.subplots(1, 2, figsize=(12,4))
    
    # Histogram of Steps
    sns.histplot(steps, kde=True, color='skyblue', ax=axs[0])
    axs[0].set_title("Steps Distribution")
    
    # Steps vs Calories Scatter
    sns.scatterplot(x=steps, y=calories, color='red', ax=axs[1])
    axs[1].set_title("Steps vs Calories")
    
    fig.tight_layout()
    
    canvas = FigureCanvasTkAgg(fig, master=chart_frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=BOTH, expand=1)

def trend_analysis():
    cursor.execute("SELECT steps, calories FROM patient_health_data")
    data = cursor.fetchall()
    if not data:
        messagebox.showwarning("No Data", "No data to analyze!")
        return
    steps, calories = zip(*data)
    slope, intercept, r_value, p_value, std_err = linregress(steps, calories)
    messagebox.showinfo("Trend Analysis",
                        f"Calories vs Steps Trend:\nSlope: {slope:.2f}\nIntercept: {intercept:.2f}\nR-squared: {r_value**2:.2f}")

def update_summary():
    cursor.execute("SELECT COUNT(*), AVG(steps), AVG(calories), AVG(heart_rate), AVG(sleep_hours) FROM patient_health_data")
    data = cursor.fetchone()
    total_var.set(f"Total Patients: {data[0]}")
    avg_steps_var.set(f"Avg Steps: {data[1]:.1f}" if data[1] else "Avg Steps: 0")
    avg_cal_var.set(f"Avg Calories: {data[2]:.1f}" if data[2] else "Avg Calories: 0")
    avg_hr_var.set(f"Avg Heart Rate: {data[3]:.1f}" if data[3] else "Avg Heart Rate: 0")
    avg_sleep_var.set(f"Avg Sleep: {data[4]:.1f}" if data[4] else "Avg Sleep: 0")

# ------------------- Summary Cards -------------------
summary_frame = Frame(root, bg="#f0f4f7")
summary_frame.pack(pady=10, fill=X)

Label(summary_frame, textvariable=total_var, font=("Arial", 14, "bold"), bg="#4da6ff", width=22, pady=10).pack(side=LEFT, padx=10)
Label(summary_frame, textvariable=avg_steps_var, font=("Arial", 14, "bold"), bg="#66ff99", width=22, pady=10).pack(side=LEFT, padx=10)
Label(summary_frame, textvariable=avg_cal_var, font=("Arial", 14, "bold"), bg="#ffcc66", width=22, pady=10).pack(side=LEFT, padx=10)
Label(summary_frame, textvariable=avg_hr_var, font=("Arial", 14, "bold"), bg="#ff6666", width=22, pady=10).pack(side=LEFT, padx=10)
Label(summary_frame, textvariable=avg_sleep_var, font=("Arial", 14, "bold"), bg="#9999ff", width=22, pady=10).pack(side=LEFT, padx=10)

# ------------------- Entry Form -------------------
form_frame = Frame(root, bg="#f0f4f7", bd=2, relief=RIDGE)
form_frame.place(x=20, y=150)

labels = ["ID", "Name", "Age", "Gender", "Weight(kg)", "Height(cm)", "Steps", "Calories", "Heart Rate", "Sleep(Hours)"]
vars = [id_var, name_var, age_var, gender_var, weight_var, height_var, steps_var, calories_var, heart_rate_var, sleep_var]

for i, (label, var) in enumerate(zip(labels, vars)):
    Label(form_frame, text=label, font=("Arial", 12, "bold"), bg="#f0f4f7").grid(row=i, column=0, pady=5, sticky=W)
    Entry(form_frame, textvariable=var, font=("Arial", 12)).grid(row=i, column=1, pady=5)

# ------------------- Buttons -------------------
Button(form_frame, text="Add", command=add_patient, bg="#4caf50", fg="white", font=("Arial",12,"bold"), width=14).grid(row=0, column=2, padx=10, pady=5)
Button(form_frame, text="Update", command=update_patient, bg="#2196f3", fg="white", font=("Arial",12,"bold"), width=14).grid(row=1, column=2, padx=10, pady=5)
Button(form_frame, text="Delete", command=delete_patient, bg="#f44336", fg="white", font=("Arial",12,"bold"), width=14).grid(row=2, column=2, padx=10, pady=5)
Button(form_frame, text="Charts", command=show_charts, bg="#ff9800", fg="white", font=("Arial",12,"bold"), width=14).grid(row=3, column=2, padx=10, pady=5)
Button(form_frame, text="Trend Analysis", command=trend_analysis, bg="#9c27b0", fg="white", font=("Arial",12,"bold"), width=14).grid(row=4, column=2, padx=10, pady=5)

# ------------------- Patient Table -------------------
table_frame = Frame(root)
table_frame.place(x=500, y=150, width=870, height=350)

scroll_x = Scrollbar(table_frame, orient=HORIZONTAL)
scroll_y = Scrollbar(table_frame, orient=VERTICAL)

patient_table = ttk.Treeview(table_frame, columns=("id","name","age","gender","weight","height","steps","calories","heart_rate","sleep"),
                             xscrollcommand=scroll_x.set, yscrollcommand=scroll_y.set)
scroll_x.pack(side=BOTTOM, fill=X)
scroll_y.pack(side=RIGHT, fill=Y)
scroll_x.config(command=patient_table.xview)
scroll_y.config(command=patient_table.yview)

for col in patient_table["columns"]:
    patient_table.heading(col, text=col.title())
    patient_table.column(col, width=100)
patient_table['show'] = 'headings'
patient_table.pack(fill=BOTH, expand=1)
patient_table.bind("<ButtonRelease-1>", select_patient)

# ------------------- Embedded Chart Frame -------------------
chart_frame = Frame(root, bg="#f0f4f7", bd=2, relief=RIDGE)
chart_frame.place(x=20, y=520, width=1350, height=240)

# ------------------- Initialize -------------------
fetch_patients()
update_summary()
root.mainloop()
